<a href="https://colab.research.google.com/github/isurushanaka/ICARC2025-Tutorial/blob/main/Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **WSN based Indoor Localization**

### Preamble

In [6]:
import warnings

# Hide FutureWarning messages
warnings.filterwarnings('ignore', category=FutureWarning)

UJIIndoorLoc: [Paper Link](https://ieeexplore.ieee.org/document/7275492/similar#similar) / [Dataset Link](https://archive.ics.uci.edu/dataset/310/ujiindoorloc)

In [7]:
!wget https://archive.ics.uci.edu/static/public/310/ujiindoorloc.zip

--2025-02-10 08:07:48--  https://archive.ics.uci.edu/static/public/310/ujiindoorloc.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘ujiindoorloc.zip.1’

ujiindoorloc.zip.1      [ <=>                ]   1.40M  7.35MB/s    in 0.2s    

2025-02-10 08:07:48 (7.35 MB/s) - ‘ujiindoorloc.zip.1’ saved [1463759]



In [8]:
!unzip /content/ujiindoorloc.zip

Archive:  /content/ujiindoorloc.zip
replace UJIndoorLoc/trainingData.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: UJIndoorLoc/trainingData.csv  
replace UJIndoorLoc/validationData.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: UJIndoorLoc/validationData.csv  


# Dataset

In [9]:
training_data_path = "/content/UJIndoorLoc/trainingData.csv"
test_data_path = "/content/UJIndoorLoc/validationData.csv"

In [10]:
import pandas as pd

train_df = pd.read_csv(training_data_path)
test_df = pd.read_csv(test_data_path)
train_df

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733
1,100,100,100,100,100,100,100,100,100,100,...,100,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691
2,100,100,100,100,100,100,100,-97,100,100,...,100,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095
3,100,100,100,100,100,100,100,100,100,100,...,100,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807
4,100,100,100,100,100,100,100,100,100,100,...,100,-7632.1436,4.864982e+06,0,0,122,2,11,13,1369909710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19932,100,100,100,100,100,100,100,100,100,100,...,100,-7485.4686,4.864875e+06,3,1,1,2,18,10,1371710683
19933,100,100,100,100,100,100,100,100,100,100,...,100,-7390.6206,4.864836e+06,1,2,140,2,18,10,1371710402
19934,100,100,100,100,100,100,100,100,100,100,...,100,-7516.8415,4.864889e+06,3,1,13,2,18,10,1371710921
19935,100,100,100,100,100,100,100,100,100,100,...,100,-7537.3219,4.864896e+06,3,1,113,2,18,10,1371711049


# Data Preprocessing

### 1. Label Encoding

In [11]:
# new unique identifier by combining these two columns (BUILDINGID, FLOOR)

train_df['Unique_SPACEID'] = train_df['BUILDINGID'].astype(str) + '_' + train_df['FLOOR'].astype(str)
test_df['Unique_SPACEID'] = test_df['BUILDINGID'].astype(str) + '_' + test_df['FLOOR'].astype(str)

In [12]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder() # Initialize the encoder

encoder.fit(train_df['Unique_SPACEID']) # Fit the encoder on training data

# Transform training data
train_df['Unique_SPACEID'] = encoder.transform(train_df['Unique_SPACEID'])

# Transform validation data
# Assign -1 for unknown labels
test_df['Unique_SPACEID'] = test_df['Unique_SPACEID'].apply(lambda x:encoder.transform([x])[0] if x in encoder.classes_ else -1)

In [13]:
train_df

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP,Unique_SPACEID
0,100,100,100,100,100,100,100,100,100,100,...,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733,6
1,100,100,100,100,100,100,100,100,100,100,...,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691,6
2,100,100,100,100,100,100,100,-97,100,100,...,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095,6
3,100,100,100,100,100,100,100,100,100,100,...,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807,6
4,100,100,100,100,100,100,100,100,100,100,...,-7632.1436,4.864982e+06,0,0,122,2,11,13,1369909710,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19932,100,100,100,100,100,100,100,100,100,100,...,-7485.4686,4.864875e+06,3,1,1,2,18,10,1371710683,7
19933,100,100,100,100,100,100,100,100,100,100,...,-7390.6206,4.864836e+06,1,2,140,2,18,10,1371710402,9
19934,100,100,100,100,100,100,100,100,100,100,...,-7516.8415,4.864889e+06,3,1,13,2,18,10,1371710921,7
19935,100,100,100,100,100,100,100,100,100,100,...,-7537.3219,4.864896e+06,3,1,113,2,18,10,1371711049,7


In [14]:
print(train_df['Unique_SPACEID'].nunique())
print(test_df['Unique_SPACEID'].nunique())

13
13


### 2. Data Normalization

* Using Min-Max Normalization (Scale to [0,1]): Min-Max scaling ensures that all values are transformed to a range between 0 and 1.

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # Initialize the scaler

# Apply StandardScaler to the RSSI columns of the train data
train_df.iloc[:, :520] = scaler.fit_transform(train_df.iloc[:, :520])

# Transform validation data using the same scaler
test_df.iloc[:, :520] = scaler.transform(test_df.iloc[:, :520])

* Using Standardization (Zero Mean, Unit Variance): Standardization scales values to have a mean of 0 and standard deviation of 1.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # Initialize the scaler

# Apply StandardScaler to the first 520 columns
train_df.iloc[:, :520] = scaler.fit_transform(train_df.iloc[:, :520])

# Transform test data using the same scaler
test_df.iloc[:, :520] = scaler.transform(test_df.iloc[:, :520])

In [16]:
train_df

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP,Unique_SPACEID
0,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733,6
1,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691,6
2,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,-5.780754,0.175093,0.066191,...,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095,6
3,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807,6
4,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,-7632.1436,4.864982e+06,0,0,122,2,11,13,1369909710,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19932,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,-7485.4686,4.864875e+06,3,1,1,2,18,10,1371710683,7
19933,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,-7390.6206,4.864836e+06,1,2,140,2,18,10,1371710402,9
19934,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,-7516.8415,4.864889e+06,3,1,13,2,18,10,1371710921,7
19935,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,-7537.3219,4.864896e+06,3,1,113,2,18,10,1371711049,7


In [17]:
X_train, y_train = train_df.iloc[:, :520], train_df["Unique_SPACEID"]
X_test, y_test = test_df.iloc[:, :520], test_df["Unique_SPACEID"]

In [18]:
X_train

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520
0,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,0.29991,0.026499,0.078752,0.079722,0.054926,0.422218,0.541602,0.033231,0.012267,0
1,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,0.29991,0.026499,0.078752,0.079722,0.054926,0.422218,0.541602,0.033231,0.012267,0
2,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,-5.780754,0.175093,0.066191,...,0.29991,0.026499,0.078752,0.079722,0.054926,0.422218,0.541602,0.033231,0.012267,0
3,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,0.29991,0.026499,0.078752,0.079722,0.054926,0.422218,0.541602,0.033231,0.012267,0
4,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,0.29991,0.026499,0.078752,0.079722,0.054926,0.422218,0.541602,0.033231,0.012267,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19932,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,0.29991,0.026499,0.078752,0.079722,0.054926,0.422218,0.541602,0.033231,0.012267,0
19933,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,0.29991,0.026499,0.078752,0.079722,0.054926,0.422218,0.541602,0.033231,0.012267,0
19934,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,0.29991,0.026499,0.078752,0.079722,0.054926,0.422218,0.541602,0.033231,0.012267,0
19935,0.03006,0.030884,0,0,0.044834,0.125136,0.172437,0.187211,0.175093,0.066191,...,0.29991,0.026499,0.078752,0.079722,0.054926,0.422218,0.541602,0.033231,0.012267,0


In [19]:
y_train

,Unique_SPACEID
0,6
1,6
2,6
3,6
4,0
...,...
19932,7
19933,9
19934,7
19935,7


# Machine Learning Models

| Model	                          |Pros	                                     |Cons                                     |
| :-------------------------------|:-----------------------------------------|:----------------------------------------|
| Logistic Regression	          |Simple, interpretable	                 |Assumes linear separability              |
| k-NN	                          |No training phase, easy to implement	     |Slow for large datasets                  |
| Decision Tree	                  |Handles complex patterns, interpretable   |Prone to overfitting                     |
| Naïve Bayes	                  |Works well for text classification	     |Assumes feature independence             |
| Random Forest                   |More robust and less prone to overfitting |Computationally expensive                |
| SVM	                          |Good for high-dimensional data	         |Slow on large datasets                   |

# Training

### 1. Logistic Regression
* A linear model for binary / multiclass classification.
* Works best when data is linearly separable.

In [20]:
from sklearn.linear_model import LogisticRegression
LGR = LogisticRegression(max_iter=100)
LGR.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### 2. k-Nearest Neighbors (k-NN)
* A non-parametric model that classifies a data point based on the majority class of its k nearest neighbors.
* Sensitive to noise and computationally expensive for large datasets.

In [21]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=5)
KNN.fit(X_train, y_train)

KNeighborsClassifier()

### 3. Decision Tree
* A tree-based model that makes decisions based on feature splits.
* Can overfit easily, so pruning or setting a maximum depth is recommended.

In [22]:
from sklearn.tree import DecisionTreeClassifier
DTC = DecisionTreeClassifier(max_depth=10)
DTC.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=10)

### 4. Naïve Bayes
* A probabilistic model based on Bayes' theorem.
* Works best for text classification and when features are independent.

In [23]:
from sklearn.naive_bayes import GaussianNB
NBC = GaussianNB()
NBC.fit(X_train, y_train)

GaussianNB()

### 5. Random Forest Classifier
* An ensemble learning method that combines multiple decision trees to improve classification accuracy and reduce overfitting.
* Works by creating multiple decision trees during training and averaging their predictions (majority voting for classification).
* More robust and less prone to overfitting than a single decision tree.

In [24]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=100)
RFC.fit(X_train, y_train)

RandomForestClassifier()

### 6. Support Vector Machine (SVM)
* Finds the best hyperplane to separate classes.
* Uses kernel trick to handle non-linear classification.
* Works well with high-dimensional data.

In [25]:
from sklearn.svm import SVC
SVM = SVC(kernel='linear')
SVM.fit(X_train, y_train)

SVC(kernel='linear')

# Evaluation

#### 1. Accuracy
$$
\text{Accuracy} = \frac{\text{True Positives} + \text{True Negatives}}{\text{Total Predictions}}
$$
* Accuracy measures the proportion of correct predictions (both true positives and true negatives) out of the total predictions made

#### 2. Precision
$$
\text{Precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}
$$
* Precision measures the proportion of true positive predictions out of all positive predictions

#### 3. Recall
$$
\text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}
$$
* Recall (also known as Sensitivity or True Positive Rate) measures the proportion of true positive predictions out of all actual positive instances

#### 4. F1 Score
$$
\text{F1 Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$
* The F1 score is the harmonic mean of precision and recall, providing a single metric that balances both

In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### 1. Logistic Regression

In [27]:
y_pred = LGR.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy : {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall   : {recall:.4f}')
print(f'F1 Score : {f1:.4f}')

Accuracy : 0.7471
Precision: 0.7797
Recall   : 0.7471
F1 Score : 0.7491


### 2. k-Nearest Neighbors (k-NN)

In [28]:
y_pred = KNN.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy : {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall   : {recall:.4f}')
print(f'F1 Score : {f1:.4f}')

Accuracy : 0.6562
Precision: 0.7169
Recall   : 0.6562
F1 Score : 0.6602


### 3. Decision Tree

In [29]:
y_pred = DTC.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy : {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall   : {recall:.4f}')
print(f'F1 Score : {f1:.4f}')

Accuracy : 0.6184
Precision: 0.7454
Recall   : 0.6184
F1 Score : 0.6251


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 4. Naïve Bayes

In [30]:
y_pred = NBC.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy : {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall   : {recall:.4f}')
print(f'F1 Score : {f1:.4f}')

Accuracy : 0.3645
Precision: 0.4523
Recall   : 0.3645
F1 Score : 0.3372


### 5. Random Forest Classifier

In [31]:
y_pred = RFC.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy : {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall   : {recall:.4f}')
print(f'F1 Score : {f1:.4f}')

Accuracy : 0.8704
Precision: 0.8949
Recall   : 0.8704
F1 Score : 0.8717


### 6. Support Vector Machine (SVM)

In [32]:
y_pred = SVM.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy : {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall   : {recall:.4f}')
print(f'F1 Score : {f1:.4f}')

Accuracy : 0.7381
Precision: 0.7706
Recall   : 0.7381
F1 Score : 0.7398


# Graphs

In [44]:
def get_buildingid_and_floor(unique_space_ids, test=False):
  if test:

  unique_space_ids = encoder.inverse_transform([unique_space_ids])
  building_floor_strings = encoder.inverse_transform(unique_space_ids)
  building_id, floor = building_floor_strings.split('_')


In [45]:
get_buildingid_and_floor(y_pred, test=True)

AttributeError: 'numpy.ndarray' object has no attribute 'type'

# Special Models

## 1. SVM
* Train SVM models with different kernels (linear, poly, rbf, sigmoid)

In [33]:
from sklearn.svm import SVC
kernels = ['linear', 'poly', 'rbf', 'sigmoid'] # List of kernels
results = {} # Dictionary to store evaluation metrics

In [34]:
for kernel in kernels:
    print(f"Training SVM with {kernel} kernel...")

    model = SVC(kernel=kernel, random_state=42) # Initialize SVM model
    model.fit(X_train, y_train) # Train the model
    y_pred = model.predict(X_test) # Make predictions

    # Compute evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Store results
    results[kernel] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1
    }

    print(f"Kernel: {kernel} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")


Training SVM with linear kernel...
Kernel: linear - Accuracy: 0.7381, Precision: 0.7706, Recall: 0.7381, F1-score: 0.7398
Training SVM with poly kernel...
Kernel: poly - Accuracy: 0.7012, Precision: 0.7651, Recall: 0.7012, F1-score: 0.6971
Training SVM with rbf kernel...
Kernel: rbf - Accuracy: 0.4887, Precision: 0.7564, Recall: 0.4887, F1-score: 0.5126
Training SVM with sigmoid kernel...
Kernel: sigmoid - Accuracy: 0.7291, Precision: 0.7592, Recall: 0.7291, F1-score: 0.7334


## 2. Neural Network

In [35]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the Neural Network Model
model = keras.Sequential([
    layers.Dense(512, activation="relu", input_shape=(520,)),  # Input Layer
    layers.Dense(256, activation="relu"),  # Hidden Layer
    layers.Dense(128, activation="relu"),  # Hidden Layer
    layers.Dense(len(np.unique(y_train)), activation="softmax")  # Output Layer
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/50
624/624 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.8132 - loss: 0.5408 - val_accuracy: 0.5203 - val_loss: 6.2466
Epoch 2/50
624/624 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9532 - loss: 0.1294 - val_accuracy: 0.5356 - val_loss: 6.4321
Epoch 3/50
624/624 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9606 - loss: 0.1113 - val_accuracy: 0.5734 - val_loss: 4.3299
Epoch 4/50
624/624 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9673 - loss: 0.1054 - val_accuracy: 0.5608 - val_loss: 5.1734
Epoch 5/50
624/624 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9796 - loss: 0.0620 - val_accuracy: 0.5482 - val_loss: 4.9148
Epoch 6/50
624/624 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9722 - loss: 0.0999 - val_accuracy: 0.6085 - val_loss: 3.5638
Epoch 7/50
624/624 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9751 - loss: 0.0688 - val_accuracy: 0.5509 - val_loss: 4.9700
Epoch 8/50
624/624 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9826 - loss: 0.0527 - val_accurac

In [ ]:
# Predict on test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels

# Compute Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average="weighted")
recall = recall_score(y_test, y_pred_classes, average="weighted")
f1 = f1_score(y_test, y_pred_classes, average="weighted")

# Print results
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy : 0.6175
Precision: 0.6874
Recall   : 0.6175
F1 Score : 0.6269
